Source: http://andyfiedler.com/2016/03/using-jupyter-for-exploring-a-heroku-database

In [2]:
import sqlalchemy as db
from sodapy import Socrata

import pandas as pd

import subprocess
import json

# Data
---

### DHS Daily Report
Source: https://data.cityofnewyork.us/Social-Services/DHS-Daily-Report/k46n-sa2m

This dataset includes the daily number of families and individuals residing in the Department of Homeless Services (DHS) shelter system and the daily number of families applying to the DHS shelter system.

In [2]:
creds_file = open('../creds.json', 'r')
socrata_creds = json.loads(creds_file.read())

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", socrata_creds['app_token'])

In [3]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("k46n-sa2m", limit = 10_000);

# Convert to pandas DataFrame
daily_df = pd.DataFrame.from_records(results);

In [4]:
results[0]

{'date_of_census': '2019-11-23T00:00:00.000',
 'total_adults_in_shelter': '38393',
 'total_children_in_shelter': '21747',
 'total_individuals_in_shelter': '60140',
 'single_adult_men_in_shelter': '12217',
 'single_adult_women_in_shelter': '4598',
 'total_single_adults_in_shelter': '16815',
 'families_with_children_in_shelter': '12208',
 'adults_in_families_with_children_in_shelter': '16325',
 'children_in_families_with_children_in_shelter': '21747',
 'total_individuals_in_families_with_children_in_shelter_': '38072',
 'adult_families_in_shelter': '2493',
 'individuals_in_adult_families_in_shelter': '5253'}

# Create table in database
Source: https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91

In [3]:
creds_file = open('./database_cred.json', 'r')
database_creds = json.loads(creds_file.read())

In [4]:
engine = db.create_engine(database_creds['url'])

connection = engine.connect()
metadata = db.MetaData()

In [114]:
for col in daily_df.columns:
    

Index(['date_of_census', 'total_adults_in_shelter',
       'total_children_in_shelter', 'total_individuals_in_shelter',
       'single_adult_men_in_shelter', 'single_adult_women_in_shelter',
       'total_single_adults_in_shelter', 'families_with_children_in_shelter',
       'adults_in_families_with_children_in_shelter',
       'children_in_families_with_children_in_shelter',
       'total_individuals_in_families_with_children_in_shelter_',
       'adult_families_in_shelter',
       'individuals_in_adult_families_in_shelter'],
      dtype='object')

In [109]:
emp = db.Table('ny_dhs_daily', metadata,
              db.Column('date_of_sensus', db.Date()),
              db.Column('total_adults_in_shelter', db.Integer()),
              extend_existing=True)
metadata.create_all(engine) #Creates the table

In [110]:
inspector = db.inspect(engine)
inspector.get_columns('ny_dhs_daily')

[{'name': 'date_of_sensus',
  'type': DATE(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'total_adults_in_shelter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'total_children_in_shelter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

In [124]:
daily_df.columns[4]

'single_adult_men_in_shelter'

In [122]:
emp = db.Table('ny_dhs_daily', metadata,
              db.Column(daily_df.columns[4], db.Integer()),
              db.Column(daily_df.columns[5], db.Integer()),
              extend_existing=True)
metadata.create_all(engine) #Creates the table

In [123]:
inspector = db.inspect(engine)
inspector.get_columns('ny_dhs_daily')

[{'name': 'date_of_sensus',
  'type': DATE(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'total_adults_in_shelter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'total_children_in_shelter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

# Return full table

In [137]:
metadata.tables['ny_dhs_daily']

Table('ny_dhs_daily', MetaData(bind=None), Column('date_of_sensus', Date(), table=<ny_dhs_daily>), Column('total_adults_in_shelter', Integer(), table=<ny_dhs_daily>), Column('single_adult_men_in_shelter', Integer(), table=<ny_dhs_daily>), Column('single_adult_women_in_shelter', Integer(), table=<ny_dhs_daily>), schema=None)

In [89]:
cols = []
for elem in inspector.get_columns('ny_dhs_daily'):
    cols.append(elem['name'])

In [90]:
to_insert = db.Table('ny_dhs_daily', metadata,
              db.Column(daily_df.columns[4], db.Integer()),
              db.Column(daily_df.columns[5], db.Integer()),
              extend_existing=True)

In [112]:
#Inserting many records at ones
query = db.insert(emp)

values_list = [{cols[0]:'2019-11-01', cols[1]:2, cols[2]:3}]


ResultProxy = connection.execute(emp, values_list)

ObjectNotExecutableError: Not an executable object: Table('ny_dhs_daily', MetaData(bind=None), Column('date_of_sensus', Date(), table=<ny_dhs_daily>), Column('total_adults_in_shelter', Integer(), table=<ny_dhs_daily>), schema=None)

In [103]:
inspector = db.inspect(engine)
inspector.get_columns('ny_dhs_daily')

[{'name': 'date_of_sensus',
  'type': DATE(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'total_adults_in_shelter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'total_children_in_shelter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

# Dropping column

In [132]:
inspector = db.inspect(engine)
inspector.get_columns('ny_dhs_daily')

[{'name': 'total_children_in_shelter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

In [131]:
a = 'total_adults_in_shelter'
sql = f"""
ALTER TABLE ny_dhs_daily
DROP {a}
"""
connection.execute(sql)

# Dropping table

In [40]:
sql = """
DROP TABLE ny_dhs_daily
"""

connection.execute(sql)

In [41]:
result